### Demonstration of Monte Carlo Simulator 

#### Import library

In [ ]:
import sys
from os import getcwd , path

# Calculate the path to the directory two levels up
parent_directory = path.abspath(path.join(path.dirname(getcwd()),'..'))

# Append the module's directory to sys.path if it's not already included
module_directory = path.join(parent_directory, 'modules')
if module_directory not in sys.path:
    sys.path.insert(0, module_directory)

# import required classes from modules folder
from base import PriceData , MonteCarloSimulator

#### Load Nifty price data, plot time series and sequence of returns

In [ ]:
# instantiate object that holds historic price data with instrument name, starting and ending dates
# daily price data resampled to monthly timeframe, with closing price on last day of month used
nifty = PriceData(instrument="^NSEI", start_date="2007-01-01", end_date="2023-11-01", re_sample="M")
# use 'plot' method of object to plot historic price and returns evolution
nifty.plot()

#### Instantiate Monte Carlo Simulator object for Nifty price series

In [ ]:
# simulator object can be created directly from the PriceData object
nifty_sim = MonteCarloSimulator(nifty)

#### Simulate MC paths and plot final results

In [ ]:
# fraction of total corpus to invest in primary instrument
# implies re-balancing between primary and secondary instrument at beginning of each period
FRACTION = 0.8
# number of paths to simulate for Monte Carlo simulation
NPATHS = 10000
# number of periods to simulate : 5 years * 12 months/year => 5 years of total time 
NPERIODS = int(12*5)

# function : returns of primary instrument -> return of secondary instrument 
# e.g. insurance contract can have 10x return if primary instrument goes down by 20% 
# e.g. in this case, secondary instrument is a fixed income instrument with annual return of 7% 
holding_reward_func = lambda x: 1.0 + (0.07/12.0)

# the simulator object is callable, and it carries out a simple MC simulation
# follows Geometric Brownian Motion (random walk of log returns), autocorrelation is delta function 
nifty_sim(FRACTION, holding_reward_func, n_paths=NPATHS, n_periods=NPERIODS)

# once simulation is done, all paths and final distribution of returns are plotted 
nifty_sim.plot_paths()

##### Compute CAGR and spread of final returns

In [ ]:
# helper function to convert final returns to CAGR in percentage
cagr = lambda x,y: f"Median CAGR = {((x**(1.0/y)) - 1.0)*100.0:.2f}%"  

# helper function to compute "spread" of final returns distrubtion
# simple choice is expressing standard deviation in percentage of mean 
spread = lambda x,y: f"Spread (Std/Median) = {(x / y)*100.0:.1f}%"

# uses methods of simulator object to get statitical properties from end of simulation period distribution
print( cagr(nifty_sim.compute_median_return(),5.0) )
print( spread(nifty_sim.compute_std_return() , nifty_sim.compute_median_return()))

#### Let's try a different fraction : 20% primary instrument , 80% fixed return instrument

In [ ]:
# fraction of total corpus to invest in primary instrument
# implies re-balancing between primary and secondary instrument at beginning of each period
FRACTION = 0.2
# number of paths to simulate for Monte Carlo simulation
NPATHS = 10000
# number of periods to simulate : 5 years * 12 months/year => 5 years of total time 
NPERIODS = int(12*5)

# function : returns of primary instrument -> return of secondary instrument 
# e.g. insurance contract can have 10x return if primary instrument goes down by 20% 
# e.g. in this case, secondary instrument is a fixed income instrument with annual return of 7% 
holding_reward_func = lambda x: 1.0 + (0.07/12.0)

# the simulator object is callable, and it carries out a simple MC simulation
# follows Geometric Brownian Motion (random walk of log returns), autocorrelation is delta function 
nifty_sim(FRACTION, holding_reward_func, n_paths=NPATHS, n_periods=NPERIODS)

# once simulation is done, all paths and final distribution of returns are plotted 
nifty_sim.plot_paths()

#### Let's compare the end of period returns in this case ! 

In [ ]:
# uses methods of simulator object to get statitical properties from end of simulation period distribution
print( cagr(nifty_sim.compute_median_return(),5.0) )
print( spread(nifty_sim.compute_std_return() , nifty_sim.compute_median_return()))

#### Some takeaways
- Although the $80:20$ (nifty index : fixed income) bet sizing has slightly higher median returns (CAGR of $8.9\%$ instead of $7.8\%$) than the $20:80$ case, the returns distribution has significantly higher spread ($42\%$ vs $9\%$).

- A starker constrast appears when looking at the worst outcomes i.e. bottom 5 percentile paths, where the $80:20$ case loses $20\%$ at the end of 5 years, compared to the gain of $20\%$ for the $20:80$ case. 

- Optimal "bet" size is not quite obvious, trade-offs exist i.e. slightly higher median CAGR comes at the cost of significantly worse spread as well as worst case outcomes (bottom 5 percentile paths). 